In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sps

import seaborn as sns
from matplotlib import pyplot as plt

import plotly.graph_objs as go
sns.set_style("darkgrid")

### 1. Реализовать формулу подсчета длительности теста, сравнить ее с онлайн калькуляторами (например https://mindbox.ru/tools/ab-test-calculator/ ). При сравнении оценить мощность критерия при указанном изменении и рассчитанном количестве наблюдений в выборке

In [57]:
def get_power(p_values, alpha=0.05):
    """Оценка мощности критерия, при условии, что значения p_value взяты при наличии 
    различий в сравниваемых выборках 
    """
    p_values = np.array(p_values)
    return p_values[p_values < alpha].shape[0] / p_values.shape[0] * 100

In [58]:
def get_qq_plot(p_values):
    """Рисует распределение p-value"""
    p_values = np.array(p_values)
    probs = []
    x = [0.01 * i for i in range(101)]
    for i in range(101):
        alpha_step = 0.01 * i
        probs.append(p_values[p_values < alpha_step].shape[0] / p_values.shape[0])
    fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name="p_value"),
                 go.Scatter(x=x, y=x, mode="lines", name="uniform")])
    fig.update_layout(height=600, width=600, title="Q-Q plot") 
    return fig

In [5]:
def duration(k, delta_effect, sigma_1, sigma_2, alpha=0.05, beta=0.2):
    z = sps.norm.ppf(1 - alpha/2) + sps.norm.ppf(1-beta)
    n = (k + 1) * z ** 2 * (sigma_1 ** 2 + sigma_2 ** 2 / k) / (delta_effect ** 2)
    return n

In [21]:
def calc_sigma(p, effect):
    s1 = np.sqrt(p * (1 - p))
    s2 = np.sqrt((p + effect) * (1 - p - effect))
    return s1, s2

In [28]:
alpha = 0.05

In [32]:
p = 0.25
effect = 0.1
beta = 0.2
sigma_1, sigma_2 =  calc_sigma(p, effect)


print('effect:', effect)
print('power:', 1 - beta )
print('alpha:', alpha)
print('duration:',
       int(duration(k=1, delta_effect=effect, sigma_1=sigma_1, sigma_2=sigma_2, alpha=alpha, beta=beta))
       )

effect: 0.1
power: 0.8
alpha: 0.05
duration: 651


In [30]:
p = 0.5
effect = 0.25
beta = 0.25
sigma_1, sigma_2 =  calc_sigma(p, effect)

print('effect:', effect)
print('power:', 1 - beta )
print('alpha:', alpha)
print('duration:',
       int(duration(k=1, delta_effect=effect, sigma_1=sigma_1, sigma_2=sigma_2, alpha=alpha, beta=beta))
       )

effect: 0.25
power: 0.75
alpha: 0.05
duration: 97


In [31]:
p = 0.5
effect = 0.2
beta = 0.1
sigma_1, sigma_2 =  calc_sigma(p, effect)

print('effect:', effect)
print('power:', 1 - beta )
print('alpha:', alpha)
print('duration:',
       int(duration(k=1, delta_effect=effect, sigma_1=sigma_1, sigma_2=sigma_2, alpha=alpha, beta=beta))
       )

effect: 0.2
power: 0.9
alpha: 0.05
duration: 241


Вывод: результат не отличается от подсчитанного калькулятором (см изображения в репозитории)

#### 2. Реализовать метод линеаризации. Проверить для него корректность и мощность. Мощность должна быть больше, чем просто на обычных значениях конверсии пользователей.

In [50]:
def linearization(loc_1, loc_2, scale_1, scale_2):
    n_exp = 1000
    p_values = []
    p_values_lin = []
    
    for _ in range(n_exp):
        records = []
        for i in range(100):
            n_views = int(sps.expon.rvs(loc=loc_1, scale=scale_1))
            clicks = sps.bernoulli.rvs(p=0.05, size=n_views)
            records.append([n_views, np.sum(clicks), np.sum(clicks)/ n_views, "A"])
        for i in range(100):
            n_views = int(sps.expon.rvs(loc=loc_2, scale=scale_2))
            clicks = sps.bernoulli.rvs(p=0.05, size=n_views)
            records.append([n_views, np.sum(clicks), np.sum(clicks)/ n_views, "B"])
        df_data = pd.DataFrame(records, columns=["views", "clicks", "cr", "group"])
        
        cr_A = df_data[df_data["group"] == "A"]["clicks"].sum() / df_data[df_data["group"] == "A"]["views"].sum()
        df_data["cr_lin"] = df_data["clicks"] - cr_A * df_data["views"]

        x_a = df_data[df_data["group"] == "A"]["cr"]
        x_b = df_data[df_data["group"] == "B"]["cr"]
        p_value = sps.ttest_ind(x_a, x_b).pvalue
        p_values.append(p_value)
        
        x_a_lin = df_data[df_data["group"] == "A"]["cr_lin"]
        x_b_lin = df_data[df_data["group"] == "B"]["cr_lin"]
        p_value_lin = sps.ttest_ind(x_a_lin, x_b_lin).pvalue
        p_values_lin.append(p_value_lin)
        
    fig1 = get_qq_plot(p_values)
    fig2 = get_qq_plot(p_values_lin)
    return p_values, p_values_lin, fig1, fig2


In [127]:
p_values, p_values_lin, fig1, fig2 = linearization(1, 1, 1, 1)
print('Мощность: ', get_power(p_values, alpha=0.05))
print('Мощность линеаризация: ', get_power(p_values_lin, alpha=0.05))

Мощность:  5.7
Мощность линеаризация:  4.3999999999999995


In [128]:
fig1

In [39]:
fig2

In [129]:
p_values, p_values_lin, fig1, fig2 = linearization(1, 10, 1, 1)
print('Мощность: ', get_power(p_values, alpha=0.05))
print('Мощность линеаризация: ', get_power(p_values_lin, alpha=0.05))

Мощность:  8.6
Мощность линеаризация:  42.199999999999996


In [130]:
fig1

In [131]:
fig2

In [132]:
p_values, p_values_lin, fig1, fig2 = linearization(1, 1, 1, 10)
print('Мощность: ', get_power(p_values, alpha=0.05))
print('Мощность линеаризация: ', get_power(p_values_lin, alpha=0.05))

Мощность:  6.9
Мощность линеаризация:  43.3


In [133]:
fig1

In [134]:
fig2

In [135]:
p_values, p_values_lin, fig1, fig2 = linearization(1, 10, 1, 10)
print('Мощность: ', get_power(p_values, alpha=0.05))
print('Мощность линеаризация: ', get_power(p_values_lin, alpha=0.05))

Мощность:  7.1
Мощность линеаризация:  58.199999999999996


In [136]:
fig1

In [137]:
fig2

Вывод: наблюдаем увеличение мощности при использовании линеаризации

#### Реализовать метод CUPED. Проверить для него корректность и мощность. Данные на этапе до A/B тесте необходимо сгенерировать один раз, далее синтетически генерировать только часть, связанную с проведением A/B-теста.

In [84]:
def cuped(loc_1, loc_2, scale_1, scale_2, loc_prev, scale_prev, corr=True, n_exp=1000, size=10000):
    p_values = []
    p_values_cuped = []

    data = sps.norm.rvs(loc=loc_prev, scale=scale_prev, size=size)

    for i in range(n_exp):
        
        df_A = pd.DataFrame()
        df_A["user"] = [f"A_{x:5}" for x in range(size)]
        df_A["pre_exp"] = data
        df_A["payments"] = sps.norm.rvs(loc=loc_1, scale=scale_1, size=size)
        df_B = pd.DataFrame()
        df_B["pre_exp"] = data
        df_B["user"] = [f"B_{x:5}" for x in range(size)]
        df_B["payments"] = sps.norm.rvs(loc=loc_2, scale=scale_2, size=size)

        if corr:
            df_A["payments"] *= df_A["pre_exp"]
            df_B["payments"] *= df_B["pre_exp"]

        p_values.append(sps.ttest_ind(df_A["payments"], df_B["payments"]).pvalue)
        
        x_a = df_A["pre_exp"]
        x_b = df_B["pre_exp"]
        y_a = df_A["payments"]
        y_b = df_B["payments"]
        theta = np.cov(x_a, y_a)[0,1] / np.std(x_a) ** 2
        
        df_A["payments_cuped"] = df_A["payments"] - theta * df_A["pre_exp"]
        df_B["payments_cuped"] = df_B["payments"] - theta * df_B["pre_exp"]
        
        p_values_cuped.append(sps.ttest_ind(df_A["payments_cuped"], df_B["payments_cuped"]).pvalue)
    
    fig1 = get_qq_plot(p_values)
    fig2 = get_qq_plot(p_values_cuped)
    return p_values, p_values_cuped, fig1, fig2

In [100]:
p_values, p_values_cuped, fig1, fig2 = cuped(1, 1, 1, 1, 1, 1, corr=True)
print('Мощность: ', get_power(p_values, alpha=0.05))
print('Мощность CUPED: ', get_power(p_values_cuped, alpha=0.05))

Мощность:  2.0
Мощность CUPED:  5.3


In [101]:
fig1

In [102]:
fig2

In [103]:
p_values, p_values_cuped, fig1, fig2 = cuped(1, 1.05, 1, 1, 1, 1, corr=True)
print('Мощность: ', get_power(p_values, alpha=0.05))
print('Мощность CUPED: ', get_power(p_values_cuped, alpha=0.05))

Мощность:  51.800000000000004
Мощность CUPED:  69.19999999999999


In [104]:
fig1

In [105]:
fig2

In [106]:
p_values, p_values_cuped, fig1, fig2 = cuped(1, 1, 1, 10, 1, 1, corr=True)
print('Мощность: ', get_power(p_values, alpha=0.05))
print('Мощность CUPED: ', get_power(p_values_cuped, alpha=0.05))

Мощность:  5.0
Мощность CUPED:  5.1


In [107]:
fig1

In [108]:
fig2

In [113]:
p_values, p_values_cuped, fig1, fig2 = cuped(1, 1.05, 1, 1.05, 1, 1, corr=True)
print('Мощность: ', get_power(p_values, alpha=0.05))
print('Мощность CUPED: ', get_power(p_values_cuped, alpha=0.05))

Мощность:  47.5
Мощность CUPED:  64.3


In [114]:
fig1

In [115]:
fig2

In [124]:
p_values, p_values_cuped, fig1, fig2 = cuped(1, 1.05, 1, 1.05, 1, 1, corr=False)
print('Мощность: ', get_power(p_values, alpha=0.05))
print('Мощность CUPED: ', get_power(p_values_cuped, alpha=0.05))

Мощность:  92.2
Мощность CUPED:  92.2


In [125]:
fig1

In [126]:
fig2

Вывод: наличие корреляции с данными пользователей увеличивает мощность